In [ ]:
pip install lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 43.0 MB/s eta 0:00:00


In [ ]:
pip install pytorch-forecasting torch lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 8.3 MB/s eta 0:00:00


In [ ]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import yfinance as yf  # for fetching stock data
import matplotlib.pyplot as plt
import lightning.pytorch as pl
from lightning.pytorch.callbacks import EarlyStopping, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss, MAE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/all_tickers.txt', 'r') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]
print(lines)

['ADBE', 'BX', 'MET', 'DLR', 'TJX', 'TT', 'BN', 'MA', 'CPRT', 'COST', 'NOC', 'UNP', 'BKNG', 'MDLZ', 'ADI', 'WM', 'TDG', 'CI', 'SYK', 'MU', 'ABBV', 'ABNB', 'PEP', 'NOW', 'LLY', 'ISRG', 'CSCO', 'EQIX', 'HCA', 'GILD', 'VRTX', 'APH', 'VZ', 'CL', 'MSI', 'FCX', 'CDNS', 'PSX', 'BAC', 'UAL', 'CHTR', 'MDT', 'BK', 'MS', 'ROP', 'CTAS', 'MSFT', 'ZTS', 'FDX', 'RGNX', 'AMAT', 'AON', 'AZO', 'AXP', 'UBER', 'CME', 'PH', 'AMP', 'CVX', 'XOM', 'ICE', 'PCG', 'NXPI', 'KLAC', 'NVDA', 'ACN', 'SCHW', 'SQ', 'CMG', 'SHW', 'SPG', 'AMD', 'O', 'URI', 'RTX', 'PNC', 'SBUX', 'ORCL', 'TXN', 'MMM', 'TMO', 'INTU', 'DHI', 'INTC', 'AMZN', 'MNST', 'UPS', 'NEE', 'ELV', 'EMR', 'ECL', 'WELL', 'AMT', 'GE', 'PCAR', 'MCD', 'PM', 'ADSK', 'CAT', 'GS', 'GD', 'WMT 2', 'FTNT', 'PYPL', 'BDX', 'KMI', 'FICO', 'D', 'DE', 'SRE', 'TSLA', 'RCL', 'ANET', 'ABT', 'CVS', 'META', 'SO', 'COF', 'PAYX', 'COP', 'DHR', 'MCK', 'CRWD', 'HD', 'ORLY', 'WFC', 'NEM', 'FANG', 'MCO', 'DIS', 'PG', 'ALL', 'T', 'C', 'ETN', 'GWW', 'NRG', 'NSC', 'SPGI', 'NKE', 'V'

In [ ]:
def convert_data_type(df, time_cols=[], float_cols=[], cat_cols=[]):
    for col in time_cols:
        df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
    for col in float_cols:
        df[col] = df[col].astype(float)
    for col in cat_cols:
        df[col] = df[col].astype('category')
    return df

# Define the stock tickers and the date range
tickers = lines
start_date = "2024-02-03"
end_date = "2024-11-03"

In [ ]:
# Download historical data from yfinance
df = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

[*********************100%***********************]  200 of 200 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['WMT 2']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


In [ ]:
df.head()

Ticker            PCAR                                                  \
Price             Open        High         Low       Close   Adj Close   
Date                                                                     
2024-02-05  102.400002  104.370003  102.199997  104.160004  103.003952   
2024-02-06  104.629997  105.000000  103.629997  104.230003  103.073166   
2024-02-07  104.860001  105.610001  103.900002  105.059998  103.893959   
2024-02-08  105.199997  106.230003  104.629997  105.610001  104.437851   
2024-02-09  105.970001  106.080002  104.959999  106.010002  104.833420   

Ticker                      ABT                                      ...  \
Price        Volume        Open        High         Low       Close  ...   
Date                                                                 ...   
2024-02-05  2589300  112.019997  112.459999  111.150002  111.650002  ...   
2024-02-06  2120900  111.820000  114.510002  111.720001  114.000000  ...   
2024-02-07  1885200  114.599998  114.599998  112.889999  113.309998  ...   
2024-02-08  2223600  113.349998  113.349998  111.519997  112.430000  ...   
2024-02-09  1474700  112.250000  112.629997  111.190002  111.809998  ...   

Ticker             SHW                                         RTX             \
Price              Low       Close   Adj Close   Volume       Open       High   
Date                                                                            
2024-02-05  302.869995  304.489990  301.959534  2267500  91.260002  92.650002   
2024-02-06  303.890015  307.880005  305.321381  1517400  91.449997  92.669998   
2024-02-07  309.489990  309.859985  307.284912  1828500  92.580002  92.809998   
2024-02-08  307.470001  312.149994  309.555878  1260000  91.849998  92.389999   
2024-02-09  309.390015  311.739990  309.149292   912500  91.129997  91.610001   

Ticker                                                
Price             Low      Close  Adj Close   Volume  
Date                                                  
2024-02-05  90.889999  92.239998  90.123886  6764600  
2024-02-06  91.190002  92.279999  90.162979  4718000  
2024-02-07  91.820000  91.930000  89.821007  4625200  
2024-02-08  90.930000  91.040001  88.951431  6197500  
2024-02-09  90.330002  90.510002  88.433586  5948100  

[5 rows x 1200 columns]

In [ ]:
df = df.stack(level=0).reset_index()
df.columns = ['time', 'ticker', 'open', 'high', 'low', 'close', 'adjclose', 'volume']
df = convert_data_type(df, time_cols=['time'], float_cols=['open', 'high', 'low', 'close', 'volume'], cat_cols=['ticker'])

# Set the time as index and sort values by time
df = df.set_index('time', drop=True).reset_index()
df = df.groupby('ticker').apply(lambda x: x.sort_values('time')).reset_index(drop=True)


<ipython-input-10-48021d768e11>:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack(level=0).reset_index()
<ipython-input-10-48021d768e11>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = df.groupby('ticker').apply(lambda x: x.sort_values('time')).reset_index(drop=True)
<ipython-input-10-48021d768e11>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly se

In [ ]:
df

,time,ticker,open,high,low,close,adjclose,volume
0,2024-02-05,AAPL,188.149994,189.250000,185.839996,187.679993,186.765030,69668800.0
1,2024-02-06,AAPL,186.860001,189.309998,186.770004,189.300003,188.377136,43490800.0
2,2024-02-07,AAPL,190.639999,191.050003,188.610001,189.410004,188.486603,53439000.0
3,2024-02-08,AAPL,189.389999,189.539993,187.350006,188.320007,187.401917,40962000.0
4,2024-02-09,AAPL,188.649994,189.990005,188.000000,188.850006,188.169144,45155200.0
...,...,...,...,...,...,...,...,...
37568,2024-10-28,ZTS,181.529999,183.100006,180.699997,182.759995,182.327942,1909700.0
37569,2024-10-29,ZTS,182.009995,184.410004,181.039993,181.270004,180.841476,1696000.0
37570,2024-10-30,ZTS,179.919998,183.809998,179.800003,182.740005,182.307999,1957800.0
37571,2024-10-31,ZTS,180.220001,181.580002,178.740005,178.779999,178.779999,2120000.0


In [ ]:
required_df = df[['ticker','time','close']]

In [ ]:
def split_dataset(df):
    df = df.sort_values(['ticker', 'time'])
    df['time_idx'] = df.groupby('ticker').cumcount()

    # Assuming 30 days per month and 7 days per week
    total_days = df.groupby('ticker').size().max()
    test_days = 30  # One week for test
    val_days = 30  # One month for validation
    train_days = total_days - val_days - test_days

    training_data = df[df['time_idx'] < train_days]
    validation_data = df[(df['time_idx'] >= train_days) & (df['time_idx'] < train_days + val_days)]
    test_data = df[df['time_idx'] >= train_days + val_days]

    return training_data, validation_data, test_data

# Apply the function to your DataFrame
training_df, validation_df, test_df = split_dataset(required_df)

<ipython-input-13-cb966a7b6ddf>:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df['time_idx'] = df.groupby('ticker').cumcount()
<ipython-input-13-cb966a7b6ddf>:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  total_days = df.groupby('ticker').size().max()


In [ ]:
def verify_split(training_df, validation_df, test_df, expected_val_test_entries=30):
    train_counts = training_df.groupby('ticker').size()
    val_counts = validation_df.groupby('ticker').size()
    test_counts = test_df.groupby('ticker').size()

    print("Sample of training set ticker counts:")
    print(train_counts.head())
    print("\nSample of validation set ticker counts:")
    print(val_counts.head())
    print("\nSample of test set ticker counts:")
    print(test_counts.head())

    all_tickers = set(train_counts.index) | set(val_counts.index) | set(test_counts.index)

    validation_issues = []
    test_issues = []

    for ticker in all_tickers:
        if ticker in val_counts and val_counts[ticker] != expected_val_test_entries:
            validation_issues.append(f"Ticker {ticker}: {val_counts[ticker]} entries (expected {expected_val_test_entries})")
        if ticker in test_counts and test_counts[ticker] != expected_val_test_entries:
            test_issues.append(f"Ticker {ticker}: {test_counts[ticker]} entries (expected {expected_val_test_entries})")

    if validation_issues:
        print("\nValidation set issues:")
        for issue in validation_issues[:5]:  # Print first 5 issues
            print(issue)
        print(f"... and {len(validation_issues) - 5} more issues") if len(validation_issues) > 5 else None

    if test_issues:
        print("\nTest set issues:")
        for issue in test_issues[:5]:  # Print first 5 issues
            print(issue)
        print(f"... and {len(test_issues) - 5} more issues") if len(test_issues) > 5 else None

    if not validation_issues and not test_issues:
        print(f"\nVerification passed: All tickers have exactly {expected_val_test_entries} entries in both validation and test sets.")
    else:
        print(f"\nVerification failed: Some tickers do not have {expected_val_test_entries} entries in validation or test sets.")

    print(f"\nTotal tickers: {len(all_tickers)}")
    print(f"Tickers in training set: {len(train_counts)}")
    print(f"Tickers in validation set: {len(val_counts)}")
    print(f"Tickers in test set: {len(test_counts)}")

# Verify the split datasets
verify_split(training_df, validation_df, test_df)

Sample of training set ticker counts:
ticker
AAPL    129
ABBV    129
ABNB    129
ABT     129
ACN     129
dtype: int64

Sample of validation set ticker counts:
ticker
AAPL    30
ABBV    30
ABNB    30
ABT     30
ACN     30
dtype: int64

Sample of test set ticker counts:
ticker
AAPL    30
ABBV    30
ABNB    30
ABT     30
ACN     30
dtype: int64

Validation set issues:
Ticker GEV: 22 entries (expected 30)

Test set issues:
Ticker GEV: 0 entries (expected 30)

Verification failed: Some tickers do not have 30 entries in validation or test sets.

Total tickers: 199
Tickers in training set: 199
Tickers in validation set: 199
Tickers in test set: 199


<ipython-input-14-630dc7fb8abd>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train_counts = training_df.groupby('ticker').size()
<ipython-input-14-630dc7fb8abd>:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_counts = validation_df.groupby('ticker').size()
<ipython-input-14-630dc7fb8abd>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  test_counts = test_df.groupby('ticker').size()


In [ ]:
print(validation_df.shape)  # Check the shape of validation data
print(validation_df.head())  # Print the first few rows of validation data
print(validation_df['ticker'].value_counts())  # Check how many entries per ticker


(5962, 4)
    ticker       time       close  time_idx
129   AAPL 2024-08-09  216.240005       129
130   AAPL 2024-08-12  217.529999       130
131   AAPL 2024-08-13  221.270004       131
132   AAPL 2024-08-14  221.720001       132
133   AAPL 2024-08-15  224.720001       133
ticker
ZTS     30
AAPL    30
ABBV    30
ABNB    30
ABT     30
        ..
AIG     30
ALL     30
AMAT    30
AMD     30
GEV     22
Name: count, Length: 199, dtype: int64


In [ ]:
print(test_df['ticker'].value_counts())

ticker
ZTS     30
AAPL    30
ABBV    30
ABNB    30
ABT     30
        ..
AIG     30
ALL     30
AMAT    30
AMD     30
GEV      0
Name: count, Length: 199, dtype: int64


In [ ]:
max_prediction_length = 6  # One week
max_encoder_length = 15  # One month, adjust if needed

training_dataset = TimeSeriesDataSet(
    training_df,
    time_idx="time_idx",
    target="close",
    group_ids=["ticker"],
    min_encoder_length=max_encoder_length // 2,
    max_encoder_length=max_encoder_length,
    min_prediction_length=1,
    max_prediction_length=max_prediction_length,
    static_categoricals=["ticker"],
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=["close"],
    target_normalizer=GroupNormalizer(groups=["ticker"]),
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

# Create validation and test datasets
validation_dataset = TimeSeriesDataSet.from_dataset(training_dataset, validation_df, stop_randomization=True)
test_dataset = TimeSeriesDataSet.from_dataset(training_dataset, test_df, stop_randomization=True)

In [ ]:
batch_size = 128  # Increased batch size for faster training
train_dataloader = training_dataset.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation_dataset.to_dataloader(train=False, batch_size=batch_size * 2, num_workers=0)
test_dataloader = test_dataset.to_dataloader(train=False, batch_size=batch_size * 2, num_workers=0)

In [ ]:
pl.seed_everything(42)

tft = TemporalFusionTransformer.from_dataset(
    training_dataset,
    learning_rate=0.001,  # Reduced learning rate for better stability
    hidden_size=64,  # Increased hidden size for more capacity
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=32,
    loss=QuantileLoss(),
    optimizer="adamw",  # Changed to AdamW optimizer
    reduce_on_plateau_patience=4,
)

INFO: Seed set to 42
INFO:lightning.fabric.utilities.seed:Seed set to 42
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()
logger = TensorBoardLogger("lightning_logs")

In [ ]:
import torch

print("Is CUDA available:", torch.cuda.is_available())
print("Current device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

Is CUDA available: True
Current device: Tesla T4


In [ ]:
trainer = pl.Trainer(
    max_epochs=100,
    accelerator="auto",  # Colab will automatically detect and use GPU if available
    devices=1,
    enable_model_summary=True,
    gradient_clip_val=0.1,
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
    log_every_n_steps=50,
    val_check_interval=None,
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader)


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 6.2 K  | train
3  | prescalers                         | ModuleDict                      | 384    | train
4  | static_variable_selection          | VariableSelectionNetwork        | 20.9 K | train
5  | encoder_variable_selection         | VariableSelectionNetwork        | 20.7 K | train
6  | decoder_variable_selection         | VariableSelectionNetwork        | 13.7 K | train
7  | static_context_variable_selectio

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
# Print the first batch to see its structure
for batch in test_dataloader:
    print("Batch structure:", type(batch), "Length:", len(batch))
    for item in batch:
        print("Item type:", type(item), "Shape:", item.shape if hasattr(item, 'shape') else "No shape")
    break  # Just print the first batch

Batch structure: <class 'tuple'> Length: 2
Item type: <class 'dict'> Shape: No shape
Item type: <class 'tuple'> Shape: No shape


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
from pytorch_forecasting.metrics import MAE

# Perform prediction
predictions = tft.predict(test_dataloader, return_x=True)

# Determine the device
device = next(tft.parameters()).device

# Move predictions to CPU (since we'll convert to numpy later)
predicted_values = predictions.output.cpu().numpy()

# Calculate MAE for the test set
actuals = torch.cat([y[0].cpu() for x, y in iter(test_dataloader)]).numpy() # Move actuals to CPU as well

predicted_values = predicted_values.ravel()
actuals = actuals.ravel()

# Check for NaN values
nan_mask = np.isnan(predicted_values) | np.isnan(actuals)

if np.any(nan_mask):
    print("NaN values found:")
    print(f"Total NaN values: {np.sum(nan_mask)}")

    # Remove NaN values
    valid_indices = ~nan_mask
    predicted_values = predicted_values[valid_indices]
    actuals = actuals[valid_indices]

# Calculate MAE
mae_value = mean_absolute_error(actuals, predicted_values)

# Calculate MSE
mse = mean_squared_error(actuals, predicted_values)

print(f"Test MAE: {mae_value}")
print(f"Test MSE: {mse}")

# Additional information
print(f"Number of samples after removing NaNs: {len(predicted_values)}")
print(f"Predictions range: {np.min(predicted_values)} to {np.max(predicted_values)}")
print(f"Actuals range: {np.min(actuals)} to {np.max(actuals)}")

# Diagnostic information
print("\nDiagnostic Information:")
print(f"Shape of predicted_values: {predicted_values.shape}")
print(f"Shape of actuals: {actuals.shape}")
print(f"Type of predicted_values: {type(predicted_values)}")
print(f"Type of actuals: {type(actuals)}")



INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


NaN values found:
Total NaN values: 5940
Test MAE: 23.35196304321289
Test MSE: 4114.29150390625
Number of samples after removing NaNs: 36828
Predictions range: 11.059747695922852 to 3976.273193359375
Actuals range: 8.59000015258789 to 4749.10986328125

Diagnostic Information:
Shape of predicted_values: (36828,)
Shape of actuals: (36828,)
Type of predicted_values: <class 'numpy.ndarray'>
Type of actuals: <class 'numpy.ndarray'>


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
import torch
import numpy as np
from pytorch_forecasting.metrics import MAE

# Perform prediction
predictions = tft.predict(test_dataloader, return_x=True)

# Determine the device
device = next(tft.parameters()).device

# Move predictions to CPU (since we'll convert to numpy later)
predicted_values = predictions.output.cpu().numpy()

# Calculate actual values for the test set
actuals = torch.cat([y[0].cpu() for x, y in iter(test_dataloader)]).numpy()  # Move actuals to CPU as well

# Flatten predictions and actuals
predicted_values = predicted_values.ravel()
actuals = actuals.ravel()

# Check for NaN values
nan_mask = np.isnan(predicted_values) | np.isnan(actuals)

if np.any(nan_mask):
    print("NaN values found:")
    print(f"Total NaN values: {np.sum(nan_mask)}")

    # Remove NaN values
    valid_indices = ~nan_mask
    predicted_values = predicted_values[valid_indices]
    actuals = actuals[valid_indices]

# Slice for the first 4 points
predicted_4 = predicted_values[:4]
actuals_4 = actuals[:4]

# Slice for the first 14 points
predicted_14 = predicted_values[:14]
actuals_14 = actuals[:14]

# Calculate MAE and MSE for first 4 points
mae_4 = mean_absolute_error(actuals_4, predicted_4)
mse_4 = mean_squared_error(actuals_4, predicted_4)

# Calculate MAE and MSE for first 14 points
mae_14 = mean_absolute_error(actuals_14, predicted_14)
mse_14 = mean_squared_error(actuals_14, predicted_14)

# Output results
print("First 4 Points:")
print(f"MAE: {mae_4}")
print(f"MSE: {mse_4}")
print(f"Prediction Range: {predicted_4.min()} to {predicted_4.max()}")
print(f"Actual Range: {actuals_4.min()} to {actuals_4.max()}")
print("\nFirst 14 Points:")
print(f"MAE: {mae_14}")
print(f"MSE: {mse_14}")
print(f"Prediction Range: {predicted_14.min()} to {predicted_14.max()}")
print(f"Actual Range: {actuals_14.min()} to {actuals_14.max()}")

# Additional information
print("\nAdditional Diagnostic Information:")
print(f"Shape of predicted_values: {predicted_values.shape}")
print(f"Shape of actuals: {actuals.shape}")
print(f"Type of predicted_values: {type(predicted_values)}")
print(f"Type of actuals: {type(actuals)}")

def calculate_stability(predictions):
    changes = np.abs(np.diff(predictions))
    stability = np.mean(changes)
    return stability

# Calculate stability for all points
stability_all = calculate_stability(predicted_values)

print("\nAll Points:")
print(f"Prediction Stability: {stability_all}")


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


NaN values found:
Total NaN values: 5940
First 4 Points:
MAE: 4.194728851318359
MSE: 22.708974838256836
Prediction Range: 225.83145141601562 to 230.91424560546875
Actual Range: 231.3000030517578 to 233.85000610351562

First 14 Points:
MAE: 5.444142818450928
MSE: 41.17619705200195
Prediction Range: 225.83145141601562 to 233.33261108398438
Actual Range: 231.3000030517578 to 236.47999572753906

Additional Diagnostic Information:
Shape of predicted_values: (36828,)
Shape of actuals: (36828,)
Type of predicted_values: <class 'numpy.ndarray'>
Type of actuals: <class 'numpy.ndarray'>

All Points:
Prediction Stability: 6.483241081237793


In [ ]:
ticker_values = {'ADBE': 236,
 'BX': 207,
 'MET': 249,
 'DLR': 300,
 'TJX': 212,
 'TT': 26,
 'BN': 206,
 'MA': 10,
 'CPRT': 276,
 'COST': 12,
 'NOC': 18,
 'UNP': 328,
 'BKNG': 286,
 'MDLZ': 295,
 'ADI': 335,
 'WM': 14,
 'TDG': 31,
 'CI': 34,
 'SYK': 327,
 'MU': 165,
 'ABBV': 12,
 'ABNB': 16,
 'PEP': 184,
 'NOW': 203,
 'LLY': 9,
 'ISRG': 207,
 'CSCO': 193,
 'EQIX': 35,
 'HCA': 326,
 'GILD': 277,
 'VRTX': 271,
 'APH': 35,
 'VZ': 144,
 'CL': 22,
 'MSI': 34,
 'FCX': 25,
 'CDNS': 26,
 'PSX': 366,
 'BAC': 57,
 'UAL': 178,
 'CHTR': 314,
 'MDT': 256,
 'BK': 229,
 'MS': 42,
 'ROP': 285,
 'CTAS': 24,
 'MSFT': 20,
 'ZTS': 34,
 'FDX': 28,
 'RGNX': 98,
 'AMAT': 203,
 'AON': 28,
 'AZO': 283,
 'AXP': 168,
 'UBER': 121,
 'CME': 297,
 'PH': 16,
 'AMP': 426,
 'CVX': 206,
 'XOM': 8,
 'ICE': 36,
 'PCG': 262,
 'NXPI': 297,
 'KLAC': 35,
 'NVDA': 2,
 'ACN': 184,
 'SCHW': 173,
 'SQ': 11,
 'CMG': 163,
 'SHW': 408,
 'SPG': 29,
 'AMD': 101,
 'O': 336,
 'URI': 325,
 'RTX': 309,
 'PNC': 28,
 'SBUX': 140,
 'ORCL': 17,
 'TXN': 255,
 'MMM': 30,
 'TMO': 261,
 'INTU': 313,
 'DHI': 66,
 'INTC': 119,
 'AMZN': 4,
 'MNST': 433,
 'UPS': 308,
 'NEE': 260,
 'ELV': 351,
 'EMR': 333,
 'ECL': 30,
 'WELL': 38,
 'AMT': 423,
 'GE': 228,
 'PCAR': 398,
 'MCD': 341,
 'PM': 331,
 'ADSK': 276,
 'CAT': 251,
 'GS': 37,
 'GD': 30,
 'WMT 2': 8,
 'FTNT': 287,
 'PYPL': 15,
 'BDX': 24,
 'KMI': 426,
 'FICO': 369,
 'D': 386,
 'DE': 294,
 'SRE': 273,
 'TSLA': 2,
 'RCL': 174,
 'ANET': 111,
 'ABT': 271,
 'CVS': 17,
 'META': 14,
 'SO': 410,
 'COF': 374,
 'PAYX': 305,
 'COP': 277,
 'DHR': 316,
 'MCK': 27,
 'CRWD': 17,
 'HD': 20,
 'ORLY': 37,
 'WFC': 48,
 'NEM': 228,
 'FANG': 321,
 'MCO': 9,
 'DIS': 262,
 'PG': 16,
 'ALL': 245,
 'T': 299,
 'C': 46,
 'ETN': 295,
 'GWW': 38,
 'NRG': 375,
 'NSC': 306,
 'SPGI': 83,
 'NKE': 139,
 'V': 12,
 'KO': 145,
 'PANW': 194,
 'MAR': 292,
 'FI': 285,
 'GM': 121,
 'DELL': 160,
 'IBM': 276,
 'PLD': 421,
 'BSX': 163,
 'USB': 0,
 'NFLX': 7,
 'EOG': 399,
 'LIN': 348,
 'ITW': 30,
 'BMY': 220,
 'PSA': 317,
 'GEV': 29,
 'PLTR': 163,
 'AEP': 410,
 'TRV': 374,
 'CB': 380,
 'TMUS': 340,
 'JNJ': 14,
 'CRM': 55,
 'AAPL': 3,
 'PGR': 361,
 'BLK': 99,
 'WMB': 81,
 'AIG': 390,
 'TGT': 17,
 'SLB': 276,
 'SON': 253,
 'APD': 25,
 'CEG': 10,
 'HLT': 396,
 'WMT': 4,
 'LRCX': 315,
 'LMT': 174,
 'SNPS': 27,
 'GOOG': 5,
 'OKE': 386,
 'MO': 352,
 'REGN': 267,
 'AVGO': 11,
 'TFC': 154,
 'CSX': 36,
 'BA': 114,
 'AFL': 386,
 'AMGN': 268,
 'IMVT': 115,
 'UNH': 16,
 'QCOM': 122,
 'PFE': 120,
 'CARR': 330,
 'DUK': 22,
 'JCI': 328,
 'VST': 373,
 'ADP': 385,
 'RSG': 320}

total_values = sum(ticker_values.values())

# Step 2: Calculate the number of tickers
num_tickers = len(ticker_values)

# Step 3: Calculate the approximate number of values per ticker
approx_values_per_ticker = total_values / num_tickers

print(f"Approximate number of values per ticker: {approx_values_per_ticker:.2f}")

Approximate number of values per ticker: 180.53
